In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn.linear_model  as sk  # namespace conflict with pyspark
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.model_selection import KFold , cross_val_score
from sklearn.preprocessing import StandardScaler
import pandas as pd
from helper import *
import tensorflow as tf

from helper import get_news_dataset, get_data, create_classes

pd.options.display.max_columns = None
import findspark
import pyspark
from pyspark.sql import SQLContext
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.regression import LabeledPoint, LinearRegressionWithSGD
from pyspark.sql.functions import col

from pyspark.ml.linalg import VectorUDT
from pyspark.sql.functions import udf
from pyspark.ml.evaluation import RegressionEvaluator

from keras.layers import Dense
from keras.models import Model, Sequential
from keras import initializers, optimizers
from keras.utils import np_utils
from keras import backend as K


# metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, precision_score, accuracy_score 

Using TensorFlow backend.


## Import DataSet

In [2]:
sumdata_url = "https://www.dropbox.com/sh/euppz607r6gsen2/AABABUTdx7YqCeBquA1Ky7z8a/The%20SUM%20dataset?dl=1#"
news_url  = "https://www.dropbox.com/sh/euppz607r6gsen2/AACq4aMWDOIw2I_SSGqJ-r2Oa/Online%20News%20Popularity%20(Mashable%20News)?dl=1"
all_urls = [sumdata_url, news_url]

In [3]:
get_data(all_urls) # retrieves the data if there is no data folder

In [4]:
sumdata_noise_path = "data/with noise/The SUM dataset, with noise.csv"
sumdata_path = "data/without noise/The SUM dataset, without noise.csv"

## Sum Data restricted to 100,000

In [5]:
SUM_DATA_SIZE = 100000

## News restricted to 10,000

In [6]:
NEWS_DATA_SIZE = 10000

## Load datasets sum_noise

In [7]:
sumdata_noise = pd.read_csv(sumdata_noise_path, delimiter=";")
sumdata_noise = sumdata_noise.head(n=SUM_DATA_SIZE)
sumdata_noise.head(n=1)

,Instance,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5 (meaningless but please still use it),Feature 6,Feature 7,Feature 8,Feature 9,Feature 10,Noisy Target,Noisy Target Class
0,1,62485,58472,84200,86181,75529,136939,150633,230058,246491,257336,1352179,Very Large Number


## Preprocess sum_noise dataset

- Remove 'Instance' as it simply represents the row number
- Extract 'Nosiy Target' as regression target
- Extract 'Nosiy Class' as classification target
- Extract rest columns as explananatory variables
- Apply Feature Scaling to the dataset 

- Ensure all dataframe has been converted to numpy array


### Create same base line for Spark, Keras, Sklearn implementation
- Using the same feature scaled data for all implementation
- Same number iterations
- Same cost function for minimising(for gradient descent)

In [8]:
# Remove 'Instance' as it simply represents the row number

from sklearn import preprocessing
le = preprocessing.LabelEncoder()

sumdata_noise.drop('Instance', axis = 1)

# Extract 'Nosiy Target' as regression target
sumdata_noise_reg_Y = sumdata_noise.loc[:, ['Noisy Target']] 

sumdata_noise_classif_Y = sumdata_noise.loc[:, ['Noisy Target Class']]
 
sumdata_noise_classif_Y['Noisy Target Class'] = le.fit_transform(sumdata_noise_classif_Y) 
 
# Extract rest columns as explananatory variables
sumdata_noise_X =sumdata_noise.drop(['Instance','Noisy Target Class','Noisy Target'], axis = 1) 
 
sumdata_noise_classif_Y.rename(columns={'Noisy Target Class': 'label'}, inplace=True)



/home/jibin/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### Feature scaling
- Keeping them as dataframes (transform returns ndarray)

In [9]:
scX = StandardScaler()
scY = StandardScaler()
columns = sumdata_noise_X.columns

sumdata_noise_X[columns] = scX.fit_transform(sumdata_noise_X)
columns = sumdata_noise_reg_Y.columns
sumdata_noise_reg_Y[columns] = scX.fit_transform(sumdata_noise_reg_Y)


sumdata_noise_scaled = pd.concat([sumdata_noise_X,sumdata_noise_reg_Y])

## Load datasets news

In [10]:
news = get_news_dataset() 
news = news.head(n=NEWS_DATA_SIZE)
news.head(n=1)

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,kw_min_min,kw_max_min,kw_avg_min,kw_min_max,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,kw_avg_avg,self_reference_min_shares,self_reference_max_shares,self_reference_avg_sharess,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,LDA_00,LDA_01,LDA_02,LDA_03,LDA_04,global_subjectivity,global_sentiment_polarity,global_rate_positive_words,global_rate_negative_words,rate_positive_words,rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,0.0,4.680365,5.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,496.0,496.0,496.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500331,0.378279,0.040005,0.041263,0.040123,0.521617,0.092562,0.045662,0.013699,0.769231,0.230769,0.378636,0.1,0.7,-0.35,-0.6,-0.2,0.5,-0.1875,0.0,0.1875,593


In [11]:
columns= [ ' timedelta', ' n_tokens_title', ' n_tokens_content',
       ' n_unique_tokens', ' n_non_stop_words', ' n_non_stop_unique_tokens',
       ' average_token_length', ' num_keywords', ' max_positive_polarity',
       ' abs_title_sentiment_polarity', ' shares']

In [12]:
news = news[columns] 
# Classification the target values 
news['label'] = news.apply(create_classes, axis=1)

In [13]:
news_reg_Y = news[' shares'].values.reshape(-1, 1)

news_reg_X = news.iloc[:, 0:-2].values # the last 2 columns are the target values 
# no need to label encode as they already encoded
news_classif_Y = news['label'] 

scX = StandardScaler() #standardising features
scY = StandardScaler() # standardising y 
news_reg_X  = scX.fit_transform(news_reg_X)
news_reg_Y  = scX.fit_transform(news_reg_Y)
 

/home/jibin/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


In [14]:
sc = pyspark.SparkContext()
sqlContext = SQLContext(sc)


In [15]:
def spark_logistic_regression( X_train, X_test, y_train, y_test):
    """
        this uses the dataframe api
    """
    from pyspark.ml import Pipeline
    from pyspark.ml.classification import LogisticRegression
    from pyspark.ml.evaluation import MulticlassClassificationEvaluator
    # spark needs a target column
     
    
     
    X_train.reset_index(drop=True, inplace=True)
    y_train.reset_index(drop=True, inplace=True)
    
    X_test.reset_index(drop=True, inplace=True)
    y_test.reset_index(drop=True, inplace=True)
    
    train_data = pd.concat([X_train,y_train], axis =1)
    test_data  = pd.concat([X_test,y_test], axis =1)
    
 
    train_data = sqlContext.createDataFrame(train_data)
    test_data = sqlContext.createDataFrame(test_data) 
   
   
    assembler = (VectorAssembler()
        .setInputCols(train_data.columns[:-1]) # everything excluding target
        .setOutputCol("features"))
 
    lr = LogisticRegression(maxIter=10, regParam=0.01)

    pipeline = Pipeline(stages=[assembler, lr])
    model = pipeline.fit(train_data)
    
    
    predictions = model.transform(test_data) 
    evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")
    accuracy = evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"})
    precision = evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})
    return (accuracy, precision )


In [16]:
def precision(y_true, y_pred):
    """Precision metric.
    https://github.com/fchollet/keras/commit/a56b1a55182acf061b1eb2e2c86b48193a0e88f7
    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [17]:
#keras log reg
def keras1DLogReg(X_train, X_test, y_train, y_test):
    
    from sklearn import preprocessing
    encoder = preprocessing.LabelEncoder()
    
    #Reshape and normalise inputs
    X_train = X_train.as_matrix()
    X_test = X_test.as_matrix()   
    y_train = y_train.as_matrix()
    y_test  = y_test.as_matrix()
    # converting to one-hot encoding
    y_train = np_utils.to_categorical(y_train)
    y_test = np_utils.to_categorical(y_test)
    
    
    #Build the model
    output_dim = y_train.shape[1]
    model = Sequential()
    model.add(Dense(output_dim, input_dim = X_train.shape[1], activation='softmax'))
    
    batch_size = 1
    epochs = 10

    
  
    #Compile ze model
    model.compile(optimizer = 'sgd', 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy', precision ])
    history = model.fit(X_train, 
                        y_train, 
                        batch_size = batch_size,
                        epochs = epochs,
                        verbose = 0)
     
    score = model.evaluate(X_test, y_test, verbose = 0)
    
    accuracy = score[0]
    precision1= score[1]
#     print ('Test score: ', score[0])
#     print ('Test accuracy', score[1])
     
    return (accuracy,precision1)

In [18]:
def model_cross_valdiation( X, y, dataset_name, algorithm, kfolds,regression=False): 
    
    kf = KFold(n_splits=kfolds)
    rmse = np.zeros((kfolds,1))
    mae = np.zeros((kfolds,1))
    accuracy = np.zeros((kfolds,1))
    precision = np.zeros((kfolds,1))
    
    algorithm_n = algorithm.__name__ 
 

    for (i, (train_index, test_index)) in enumerate(kf.split(X)):


        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        

        # accumulating errors for each fold 
        # calculates mean across rows
        if regression:     
            rmse[i], mae[i] = algorithm( X_train, X_test, y_train, y_test)   # RMSE https://www.kaggle.com/wiki/RootMeanSquaredError
           
        else: 
            accuracy[i],precision[i] = algorithm( X_train, X_test, y_train, y_test)
           
    if regression:     
        print ("Dataset: {}\nRMSE: {}\nMAE: {}\n".format( dataset_name, rmse.mean(), mae.mean()))
    else:
        print ("Dataset: {}\Accuracy: {}\nPrecision: {}\n".format(dataset_name, accuracy.mean(), precision.mean()))
    
          

In [ ]:


model_cross_valdiation( sumdata_noise_X.head(n=100000), sumdata_noise_classif_Y.head(n=100000), "Sum data with noise", keras1DLogReg,10)


In [19]:
model_cross_valdiation( sumdata_noise_X, sumdata_noise_classif_Y, "Sum data with noise", spark_logistic_regression,10)


Dataset: Sum data with noise\Accuracy: 0.95061
Precision: 0.932101779454789



### 70/30 split 

In [21]:
def model_test_split( X, y, dataset_name, algorithm, regression=False): 
    algorithm_n = algorithm.__name__
    rmse = 0
    mae = 0 
    accuracy = 0
    precision = 0  

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

    if regression:     
        rmse, mae = algorithm( X_train, X_test, y_train, y_test)   # RMSE https://www.kaggle.com/wiki/RootMeanSquaredError
        print ("Algorithm: {} Dataset: {}\nRMSE: {}\nMAE: {}\n".format( 
                                                                algorithm_n,
                                                                dataset_name,
                                                                rmse,
                                                                mae))
        
    else: 
        accuracy ,precision  = algorithm( X_train, X_test, y_train, y_test)
        # accumulating errors for each fold
        print ("Algorithm: {} Dataset: {}\Accuracy: {}\nPrecision: {}\n".format(
                                                                algorithm_n,
                                                                dataset_name,
                                                                accuracy,
                                                                precision))



In [22]:
model_test_split( sumdata_noise_X, sumdata_noise_classif_Y, "Sum data with noise", spark_logistic_regression)

Algorithm: spark_logistic_regression Dataset: Sum data with noise\Accuracy: 0.9505666666666667
Precision: 0.9325656764017538



In [127]:
model_test_split( sumdata_noise_X, sumdata_noise_classif_Y, "Sum data with noise", keras1DLogReg)

KeyboardInterrupt: 